In [ ]:
!pip install -q tflite-model-maker

import os, numpy as np, matplotlib.pyplot as plt
import tensorflow as tf
assert tf.__version__.startswith('2'), tf.__version__

from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tflite_model_maker.config import ExportFormat, QuantizationConfig

tf.get_logger().setLevel('ERROR')
print("TF version:", tf.__version__)
print("All good ✅")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 63.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


ModuleNotFoundError: No module named 'tflite_model_maker'

In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/sidewalk_efficientdet"  # you can rename
os.makedirs(PROJECT_ROOT, exist_ok=True)
print("Project root:", PROJECT_ROOT)


In [ ]:
#@title Load dataset (EDIT paths + labels for your project)

DATA_ROOT = "/content/drive/MyDrive/sidewalk_dataset"  # TODO: change to your folder

train_images_dir = os.path.join(DATA_ROOT, "train/images")
train_ann_dir    = os.path.join(DATA_ROOT, "train/annotations")

val_images_dir   = os.path.join(DATA_ROOT, "val/images")
val_ann_dir      = os.path.join(DATA_ROOT, "val/annotations")

# 👉 EDIT this dictionary to your labels
label_map = {
    1: "sidewalk",
    2: "crosswalk",
    3: "traffic_light",
    4: "stop_sign",
    # add/remove as needed
}

print("Train images dir:", train_images_dir)
print("Val images dir:",   val_images_dir)
print("Labels:", label_map)

train_data = object_detector.DataLoader.from_pascal_voc(
    train_images_dir,
    train_ann_dir,
    label_map
)

val_data = object_detector.DataLoader.from_pascal_voc(
    val_images_dir,
    val_ann_dir,
    label_map
)

print("Train samples:", len(train_data))
print("Val samples:", len(val_data))


In [ ]:
#@title Define EfficientDet-Lite0 spec + logging directory

spec = model_spec.get('efficientdet_lite0')

# number of classes = length of label_map
spec.config.num_classes = len(label_map)

# directory where checkpoints + TensorBoard logs will go
spec.model_dir = os.path.join(PROJECT_ROOT, "efficientdet_lite0_logs")
os.makedirs(spec.model_dir, exist_ok=True)

print("Using model_dir:", spec.model_dir)
print("Num classes:", spec.config.num_classes)


In [ ]:
EPOCHS = 500  # you can lower (e.g., 20) for faster demo
BATCH_SIZE = 8

detector = object_detector.create(
    train_data,
    model_spec=spec,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    train_whole_model=True,
    validation_data=val_data,
)

print("Training done ✅")

In [ ]:
#@title Plot training vs validation loss from history (backup for report)

hist = getattr(detector, "history", None)

if hist is None:
    print("No Keras history found on detector object (version difference).")
else:
    history = hist.history
    print("History keys:", history.keys())

    # Generic loss curve
    if "loss" in history:
        plt.figure(figsize=(6,4))
        plt.plot(history["loss"], label="train_loss")
        if "val_loss" in history:
            plt.plot(history["val_loss"], label="val_loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("EfficientDet-Lite0 training loss")
        plt.legend()
        plt.grid(True)
        plt.show()


In [ ]:
#@title Launch TensorBoard (take screenshots of curves here)

%load_ext tensorboard
LOG_DIR = spec.model_dir  # this folder has /summaries inside

print("TensorBoard log dir:", LOG_DIR)
%tensorboard --logdir $LOG_DIR


In [ ]:
#@title Export EfficientDet-Lite0 to TFLite

EXPORT_DIR = os.path.join(PROJECT_ROOT, "export")
os.makedirs(EXPORT_DIR, exist_ok=True)

# Float16 quantized (good for mobile)
quant_config = QuantizationConfig.for_float16()

detector.export(
    export_dir=EXPORT_DIR,
    tflite_filename="efficientdet_lite0_sidewalk_fp16.tflite",
    label_filename="labels.txt",
    export_format=[ExportFormat.TFLITE, ExportFormat.LABEL],
    quantization_config=quant_config,
)

print("Exported files in:", EXPORT_DIR)
print(os.listdir(EXPORT_DIR))
